# Bermudan Bond Option

In this notebook we illustrate the pricing of Bermudan bond options via backward induction algorithm.

The notebook is structured as follows:

  1. Specification of a Bermudan option.

  2. Option pricing via backward induction algorithm.

  3. Numerical analysis of pricing methods:
     
     a) Density integration methods,

     b) PDE pricing method,

     c) American Monte Carlo pricing methods.

For all the tests we use the curves and Hull White model from earlier examples.

In [ ]:
import sys
sys.path.append('../') # make sure we can access the src/ folder

from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from src.hull_white_model import HullWhiteModel
from src.yieldcurve import YieldCurve

We set up initial yield curve and Hull White model.

In [ ]:
terms = [    '1y',    '2y',    '3y',    '4y',    '5y',    '6y',    '7y',    '8y',    '9y',   '10y',   '12y',   '15y',   '20y',   '25y',   '30y', '50y'   ] 
rates = [ 2.70e-2, 2.75e-2, 2.80e-2, 3.00e-2, 3.36e-2, 3.68e-2, 3.97e-2, 4.24e-2, 4.50e-2, 4.75e-2, 4.75e-2, 4.70e-2, 4.50e-2, 4.30e-2, 4.30e-2, 4.30e-2 ] 

disc_curve = YieldCurve(terms, rates)

mean_reversion = 0.03
volatility_times  = np.array([ 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, ])
volatility_values = np.array([ 90., 80., 70., 60., 50., 40., ]) * 1e-4

model = HullWhiteModel(disc_curve, mean_reversion, volatility_times, volatility_values)

## Bermudan Bond Option Specification

A bond option gives the option holder the right to buy a fixed rate bond at one or more pre-defined exercise times.

The underlying fixed rate bond pays regular coupons and redeems the bond notional at final maturity.

If the option holder decides to exercise the option then the payoff for the option holder is as follows:

  - The option holder pays the pre-defined strike price; typically equal to bond notional.

  - The option holder receives the future coupons after exercise time.

  - The option holder receives the bond notional at final maturity.

The sum of all cash paid and received flows represents the *underlying* at a given exercise time.

If there are several exercise times (i.e. a Bermudan option) then the underlying differs for each exercise time. In particular, the number of future coupons (after exercise) reduces if exercise time increases.

We consider a 10y maturity bond with unit notional paying an annual coupons of 3%.

In [ ]:
cpn = 0.03

bond = pd.DataFrame()
bond['pay_times']  = [ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 10.0 ]
bond['cash_flows'] = [ cpn, cpn, cpn, cpn, cpn, cpn, cpn, cpn, cpn,  cpn,  1.0 ]
bond

Option exercise (or option expiry) times are assumed to be in 1y, 2y, ... 9y.

In [ ]:
exercise_times = np.array([ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0 ])

Now, we need to collect the cash flows of the payoff if exercised at a given exercise time.

In [ ]:
underlying_flows = []
for T in exercise_times:
    pay_times  = np.array([ T ])     # strike price payment at exercise
    cash_flows = np.array([ -1.0 ])  # strike price equal to notional
    #
    future_times = bond['pay_times'][bond['pay_times']>T]
    future_flows = bond['cash_flows'][bond['pay_times']>T]
    #
    pay_times = np.concatenate((pay_times, future_times))
    cash_flows = np.concatenate((cash_flows, future_flows))
    underlying_flows.append({
        'pay_times' : pay_times,
        'cash_flows' : cash_flows,
    })
pd.DataFrame(underlying_flows)

## Bond Option Pricing via Backward Induction

Backward induction algorithm is implemented as a function. The function takes a *method* parameter and the details of the option instrument. The *method* parameter is an object that implements the conditional expectation calculation
$$
  V(T_0) = B(T_0) \mathbb{E}\left[ \frac{V(T_1)}{B(T_1)} \right].
$$

We use density integration method via Simpson formula for this first example.

In [ ]:
from src.methods.density_integrations import SimpsonIntegration

method = SimpsonIntegration(model)

Backward induction algorithm also requires the calculation of the underlying payoffs $U_k(x)$ at expiry time $T_k$ and for a given model state $x$. This calculation is encapsulated in a *payoff* object. In our particular example the payoff is a *CouponBond*.

In [ ]:
from src.methods.payoffs import CouponBond

underlyings = []
for T, U in zip(exercise_times, underlying_flows):
    underlyings.append(CouponBond(model, T, U['pay_times'], U['cash_flows']))

Now, we got everything to run the backward induction algorithm.

In [ ]:
from src.bermudan_option import bermudan_option_npv

berm_npv = bermudan_option_npv(exercise_times, underlyings, method, showProgress=True)
display(berm_npv)

## Numerical Analysis of Pricing Methods

Backward induction algorithm for Bermudan options can be implemented with various pricing methods. Available methods depend on the used model. For our Hull White model we implement density integration (with various variants), PDE solution and American Monte Carlo simulation. In this section we analyse the usage and accuracy of the methods.

One way of assessing the accuracy of a numerical method for Bermudan options is to apply it to a European option. Then we can use analytic formulas as a benchmark for the numerical methods.

We encapsulate European and Bermudan bond option pricing into a function to simplify comparison of our pricing methods.

In [ ]:
def europeans_and_bermudan_prices(method):
    res = []
    for T, U in zip(exercise_times, underlyings):
        res.append({
            'Exercise' : '%.0fy' % T,
            'Analytic' : model.coupon_bond_option(T, U.pay_times, U.cash_flows, 0.0, 1.0),
            'Numeric'  : bermudan_option_npv([T], [U], method),
        })
    res.append({
        'Exercise' : 'Berm',
        'Numeric'  : bermudan_option_npv(exercise_times, underlyings, method, showProgress=True),
    })
    res = pd.DataFrame(res)
    display(res)
    display(res.plot.bar(x='Exercise'))

The numerical methods need to be parametrised with discretisation parameters, e.g. number of grid points and width of the grid (measured in standard deviations). For American Monte Carlo we also need to provide simulated paths.

The choice of the parameters impacts numerical accuracy and computational effort. For practical applications it is important to carefully balance these two objectives.

Below we list the methods with some reasonable parametrisation choices. To get some intuition and experience with the methods you can change the parametrisations an check how the numerical European and Bermudan prices change.

In [ ]:
from src.methods.amc_solver import AmcSolver
from src.methods.amc_solver import AmcSolverOnlyExerciseRegression
from src.methods.amc_solver import CoterminalRateControls
from src.methods.density_integrations import DensityIntegrationWithBreakEven
from src.methods.density_integrations import CubicSplineExactIntegration
from src.methods.density_integrations import HermiteIntegration
from src.methods.density_integrations import SimpsonIntegration
from src.methods.payoffs import CouponBond
from src.methods.pde_solver import PdeSolver
from src.monte_carlo_simulation import MonteCarloSimulation

# state discretisation for density and PDE method
nGridPoints=101
stdDevs=5

# we need a MC simulation for AMC method
times = np.linspace(0.0, 20.0, 21)
n_paths = 2**16
sim = MonteCarloSimulation(model, times, n_paths, showProgress=True)
# some more AMC parametrisations
max_polynomial_degree = 2
split_ratio = 0.25
maturity = bond['pay_times'].iloc[-1]

methods = [
    CubicSplineExactIntegration(model, nGridPoints, stdDevs),
    HermiteIntegration(model, degree=5, nGridPoints=nGridPoints, stdDevs=stdDevs),
    SimpsonIntegration(model, nGridPoints, stdDevs),
    #
    DensityIntegrationWithBreakEven(CubicSplineExactIntegration(model, nGridPoints, stdDevs)),
    DensityIntegrationWithBreakEven(SimpsonIntegration(model, nGridPoints, stdDevs)),
    #
    PdeSolver(model, nGridPoints, stdDevs, theta=0.5, timeStepSize=1.0/12, lambda0N=None),
    #
    AmcSolver(sim, max_polynomial_degree, split_ratio),
    AmcSolverOnlyExerciseRegression(sim, max_polynomial_degree, split_ratio),
    AmcSolver(sim, max_polynomial_degree, split_ratio, controls=CoterminalRateControls(model, maturity)),
    AmcSolverOnlyExerciseRegression(sim, max_polynomial_degree, split_ratio, controls=CoterminalRateControls(model, maturity)),
    AmcSolver(sim, max_polynomial_degree=1, split_ratio=split_ratio, controls=CoterminalRateControls(model, maturity, strike_rate=0.0)),
]


Now we can test the calculations for the various methods considered.

In [ ]:
interact(europeans_and_bermudan_prices, method=methods)